# Notebook for Converting Sentences to Suggested ESCU Contents

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
!pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 31.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!python -m spacy download en_core_web_md
!python -m spacy download en

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.3.1/en_core_web_md-2.3.1.tar.gz#egg=en_core_web_md==2.3.1 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 MB 39.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_web_md: filename=en_core_web_md-2.3.1-py3-none-any.whl size=50916621 sha256=5607250ab4d43a6b24de62df455929256a17a80a4260f820e964004054f51552
  Stored in directory: /tmp/pip-ephem-wheel-cache-p1wch9eh/wheels/6f/37/40/51e4a94540f45642e010988c7e8a88e8e95c0364a9e4d0b672
Successfully built en_core_web_md

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notic

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import transformers
import torch
import io
import re
import requests
import pickle
import json
import spacy


from sentence_transformers import SentenceTransformer

from sklearn.decomposition import PCA
from sklearn.metrics import DistanceMetric

import matplotlib.pyplot as plt
import matplotlib as mpl

#global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

#model = SentenceTransformer('all-MiniLM-L6-V2')
#model.save(MODEL_DIRECTORY)
model = SentenceTransformer(MODEL_DIRECTORY)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

import os
for dirname, _, filenames in os.walk('/srv/notebooks/data/ESCU'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(model)

2025-05-17 12:22:06.729520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-17 12:22:06.729541: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/srv/notebooks/data/ESCU/ESCU_Dataset.csv
/srv/notebooks/data/ESCU/ESCU_DataSource.csv
/srv/notebooks/data/ESCU/.ipynb_checkpoints/ESCU_DataSource-checkpoint.csv
/srv/notebooks/data/ESCU/.ipynb_checkpoints/ESCU_Dataset-checkpoint.csv
SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


# load data

In [4]:
df = pd.read_csv("/srv/notebooks/data/ESCU/ESCU_DataSource.csv", encoding='latin-1', usecols=['id_desc', 'datasource'])

In [5]:
df.head()

,id_desc,datasource
0,"3cx_supply_chain_attack_network_indicators:The analytic provided below employs the Network_Resolution datamodel to detect domain indicators associated with the 3CX supply chain attack. By leveraging this query, you can efficiently conduct retrospective analysis of your data to uncover potential compromises.",DNS
1,7zip_commandline_to_smb_share_path:This search is to detect a suspicious 7z process with commandline pointing to SMB network share. This technique was seen in CONTI LEAK tools where it use 7z to archive a sensitive files and place it in network share tmp folder. This search is a good hunting query that may give analyst a hint why specific user try to archive a file pointing to SMB user which is un usual.,Endpoint Detection and Response
2,abnormally_high_aws_instances_launched_by_user:This search looks for AWS CloudTrail events where a user successfully launches an abnormally high number of instances. This search is deprecated and have been translated to use the latest Change Datamodel,AWS
3,abnormally_high_aws_instances_terminated_by_user:This search looks for AWS CloudTrail events where an abnormally high number of instances were successfully terminated by a user in a 10-minute window. This search is deprecated and have been translated to use the latest Change Datamodel.,AWS
4,abnormally_high_number_of_cloud_infrastructure_api_calls:This search will detect a spike in the number of API calls made to your cloud infrastructure environment by a user.,Change Events Data


# generate embeddings

In [6]:
embedding_arr = model.encode(df['id_desc'])
embedding_arr.shape

(1628, 384)

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| makeresults | eval content = "Between April and June 2024, the NullBulge group emerged targeting users in AI-centric application and gaming communities. The NullBulge persona has showcased creative methods of distributing malware targeting said tools and platforms. Though the group projects an image of activism claiming to be “protecting artists around the world” and claims to be motivated by a pro-art, anti-AI cause, rather than profit, other activities tied to this threat actor may indicate otherwise. One service the group offers is described as “payback through honeypots and malicious mods.” The group is delivering on this claim by targeting extensions and modifications of commonly used AI-art-adjacent applications and games. This has been their main area of focus recently, delivering a small variety of malware payloads. NullBulge’s attacks are characterized by ‘poisoning the well’: the group targets the software supply chain by injecting malicious code into legitimate software distribution mechanisms, exploiting trusted platforms like GitHub, Reddit and Hugging Face to maximize their reach. NullBulge announces their leaks via their own DLS/blog site, alongside occasional 4chan threads. Further, the group is using customized LockBit ransomware builds to maximize the impact of their attacks. In this post, we provide an overview of the NullBulge group’s malicious activities, and technical details of their LockBit payloads." | makemv content delim=";" | mvexpand content | fit MLTKContainer mode=stage algo=sentence_predictToESCU epochs=100 content into app:sentence_predicttoescu_model

In [7]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df_query = pd.read_csv(f)
        #full_text = f.read()
        #for l in re.split(r"(\.)", full_text):
         #   if l != ".":
          #      name.append(l + "\n")
    #pd.DataFrame(string)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    nlp = spacy.load('en_core_web_sm')
    #dfs = df.applymap(str)
    sbd = nlp.create_pipe('sentencizer')
    nlp.add_pipe(sbd)
    text = str(df_query.content.values)
    doc = nlp(text)
    sentence_list =[]
    for sentence in doc.sents:
        sentence_list.append(sentence.text)
    dfnew = pd.DataFrame({'content': sentence_list})
    return df_query, param, dfnew

In [8]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df_query, param, dfnew = stage("sentence_predicttoescu_model")
print(df_query) 
print(df_query.shape)
print(dfnew)
print(dfnew.shape)
print(str(param))

## Stage 2 - create and initialize a model

In [9]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = SentenceTransformer(MODEL_DIRECTORY)
    #return model
    #model = {}
    #model['hyperparameter'] = 42.0
    return model

In [10]:
model = init(df,param)

In [11]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


## Stage 3 - fit the model

Note that for this algorithm the model is pre-trained and therefore this stage is a placeholder only

In [12]:
# returns a fit info json object
def fit(model,df,param):
    info = {"message": "model trained"}
    return info

In [13]:
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [14]:
def apply(model,dfnew,param):
    model = SentenceTransformer(MODEL_DIRECTORY)
    escu_list_0 =[]
    escu_list_1 = []
    escu_list_2 = []
    escu_list_3 = []
    escu_list_4 = []
    df = pd.read_csv("/srv/notebooks/data/ESCU/ESCU_DataSource.csv", encoding='latin-1', usecols=['id_desc', 'datasource'])
    embedding_arr = model.encode(df['id_desc'])
    embedding_arr.shape
    for index, row in dfnew.iterrows():
        query_embedding = model.encode(str(row['content']))
        dist = DistanceMetric.get_metric('euclidean') # other distances: manhattan, chebyshev
        # compute pair wise distances between query embedding and all resume embeddings
        dist_arr = dist.pairwise(embedding_arr, query_embedding.reshape(1, -1)).flatten()
        # sort results
        idist_arr_sorted = np.argsort(dist_arr)
        escu_list_0.append(df['id_desc'].iloc[idist_arr_sorted[0]])
        escu_list_1.append(df['id_desc'].iloc[idist_arr_sorted[1]])
        escu_list_2.append(df['id_desc'].iloc[idist_arr_sorted[2]])
        escu_list_3.append(df['id_desc'].iloc[idist_arr_sorted[3]])
        escu_list_4.append(df['id_desc'].iloc[idist_arr_sorted[4]])
        #suggested_escu=df['id_desc'].iloc[idist_arr_sorted[0]]
    #print(spl_list)
    dfnew['suggested_escu_1']=escu_list_0
    dfnew['suggested_escu_2']=escu_list_1
    dfnew['suggested_escu_3']=escu_list_2
    dfnew['suggested_escu_4']=escu_list_3
    dfnew['suggested_escu_5']=escu_list_4

    return dfnew

In [15]:
returns=apply(model,dfnew,param)
print(returns)

                                                                                                                                                                                                                                                                               content  \
0                                                                                                                                                         ['Between April and June 2024, the NullBulge group emerged targeting users in AI-centric application and gaming communities.   
1                                                                                                                                                                     The NullBulge persona has showcased creative methods of distributing malware targeting said tools and platforms.   
2                                   Though the group projects an image of activism claiming to be “protecting artists around the world” and claims to be m

## Stage 5 - save the model

In [16]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # model will not be saved or reloaded as it is pre-built
    with open(MODEL_DIRECTORY + name + ".json", 'wb') as file:
        #model_dict=model.state_dict()
        #model_list=list(model)
         pickle.dump(model, file)
     #   torch.save(model.state_dict(), file)
    return model

In [17]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


## Stage 6 - load the model

In [18]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # model will not be saved or reloaded as it is pre-built
    #model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'rb') as file:
        model=pickle.load(file)
    #model = {}
     #   model = json.load(file)
    #model=model.load_model("t5","/srv/outputs/simplet5-epoch-4-train-loss-0.9567-val-loss-0.8704", use_gpu=False)
    return model

## Stage 7 - provide a summary of the model

In [19]:
# return model summary
def summary(model=None):
    returns = {"version": {"spacy": spacy.__version__} }
    if model is not None:
        s = []
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code